# Advent of Code: Day 5


## Part One

--- Day 5: If You Give A Seed A Fertilizer ---

You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```
The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

```
50 98 2
52 50 48
```
The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

```
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
```

With this map, you can look up the soil number required for each initial seed number:

- Seed number 79 corresponds to soil number 81.
- Seed number 14 corresponds to soil number 14.
- Seed number 55 corresponds to soil number 57.
- Seed number 13 corresponds to soil number 13.
- 
The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

- Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
- Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
- Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
- Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?



In [ ]:
from dataclasses import dataclass
import logging
import re
from typing import Callable, List, Dict, Tuple

In [ ]:
# Set up logging for Jupyter Notebook
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
@dataclass
class RangeMap:
    dest_range_start: int
    source_range_start: int
    range_length: int

    def get_dest_value(self, idx) -> int:
        if idx >= self.source_range_start and idx < self.source_range_start + self.range_length:
            offset: int = idx - self.source_range_start
            value: int = self.dest_range_start + offset
            # logger.debug(f'idx={idx} found in {self}')
            return value
        # FIXME: should this just return idx?
        return idx


class RangeDict:
    range_maps: List[RangeMap] = []

    def __getitem__(self, idx) -> List[int]:
        dest_values: List[int] = []
        for range_map in self.range_maps:
            dest_value = range_map.get_dest_value(idx)
            if dest_value is not None:
                dest_values.append(dest_value)
        # if len(dest_values) != 1:
        #     raise ValueError(f'Expected only one value mapped from {idx}, instead: {dest_values}')
        return dest_values
            
    def add_range_map(self, range_map: RangeMap):
        self.range_maps.append(range_map)


def parse_seed_list_str(s: str) -> List[int]:
    header, value_str = s.split(':')
    value_str = value_str.strip()
    return [int(i) for i in value_str.split(' ')]


def parse_range_str(s: str) -> RangeMap:
    numbers: List[int] = (int(i) for i in s.split(' '))
    dest_range_start: int
    source_range_start: int
    range_length: int
    dest_range_start, source_range_start, range_length = numbers
    range_map: RangeMap = RangeMap(
        dest_range_start=dest_range_start,
        source_range_start=source_range_start,
        range_length=range_length,
    )
    return range_map


def parse_input():
    seed_list: List[int] = []
    current_mode: str = None
    current_dct: RangeDict = None
    
    seed_to_soil_dct: RangeDict = RangeDict()
    soil_to_fertilizer_dct: RangeDict = RangeDict()
    fertilizer_to_water_dct: RangeDict = RangeDict()
    water_to_light_dct: RangeDict = RangeDict()
    light_to_temperature_dct: RangeDict = RangeDict()
    temperature_to_humidity_dct: RangeDict = RangeDict()
    humidity_to_location_dct: RangeDict = RangeDict()
    mode_dct_list = [
        ('seed-to-soil', seed_to_soil_dct),
        ('soil-to-fertilizer', soil_to_fertilizer_dct),
        ('fertilizer-to-water', fertilizer_to_water_dct),
        ('water-to-light', water_to_light_dct),
        ('light-to-temperature', light_to_temperature_dct),
        ('temperature-to-humidity', temperature_to_humidity_dct),
        ('humidity-to-location', humidity_to_location_dct),
    ]

    def get_mode_from_line(line: str) -> Tuple[str, RangeDict]:
        for mode, mode_dct in mode_dct_list:
            if line.startswith(mode):
                return mode, mode_dct
        return None, None
    
    with open('20231205-input.txt', 'r') as f:
        for line in f.readlines():
            line = line.strip()
            # logger.info(line)
            if line.startswith('seeds:'):
                seed_list = parse_seed_list_str(line)
                # logger.info(f'seed_list: {seed_list}')
            elif get_mode_from_line(line) != (None, None):
                current_mode, current_dct = get_mode_from_line(line)
                # logger.info(f'mode: {current_mode}')
            elif line == '':
                # logger.info('noop')
                pass
            else:
                # logger.debug(f'mode: {current_mode}, line: {line}')
                current_dct.add_range_map(parse_range_str(line))

    paths = []
    locations = []
    for seed_id in seed_list:
        # logger.info(f'seed_id: {seed_id}')
        for soil_id in seed_to_soil_dct[seed_id]:
            # logger.info(f'\tsoil_id: {soil_id}')
            for fert_id in soil_to_fertilizer_dct[soil_id]:
                # logger.info(f'\t\tfert_id: {fert_id}')
                for water_id in fertilizer_to_water_dct[fert_id]:
                    # logger.info(f'\t\t\twater_id: {water_id}')
                    for light_id in water_to_light_dct[water_id]:
                        # logger.info(f'\t\t\t\tlight_id: {light_id}')
                        for temp_id in light_to_temperature_dct[light_id]:
                            # logger.info(f'\t\t\t\t\ttemp_id: {temp_id}')
                            for hum_id in temperature_to_humidity_dct[temp_id]:
                                # logger.info(f'\t\t\t\t\t\thum_id: {hum_id}')
                                for loc_id in humidity_to_location_dct[hum_id]:
                                    # logger.info(f'\t\t\t\t\t\t\tloc_id: {loc_id}')
                                    paths.append([seed_id, soil_id, fert_id, water_id, light_id, temp_id, hum_id, loc_id])
                                    locations.append(loc_id)
    return locations

def solution_part1():
    return min(parse_input())


solution_part1()

In [ ]:
3489262449 >= 3483784779, 3489262449 < 3483784779 + 43058912 - 1

In [ ]:
3489262449 >= 3079182441, 3489262449 < 3079182441 + 511127728 - 1